To start a mini pyres

idea: what about an isabelle style [] => []



In [ ]:
from dataclasses import dataclass

@dataclass
class Term():
    pass
@dataclass
class Fn(Term):
    name: str
    args: list
@dataclass
class Var(Term):
    name: str

def subst(t : Term, s : dict[str,Term]):
    if isinstance(t, Var):
        return s.get(t.name, t)
    elif isinstance(t, Fn):
        return Fn(t.name, [subst(arg, s) for arg in t.args])
    else:
        raise ValueError("Invalid term")

def occurs_check(x : Var, t : Term):
    if isinstance(t, Var):
        return x.name == t.name
    elif isinstance(t, Fn):
        return any(occurs_check(x, arg) for arg in t.args)
    else:
        raise ValueError("Invalid term")

# https://github.com/eprover/PyRes/blob/master/unification.py
def mgu(t1:Term, t2:Term):
    l1 = [t1]
    l2 = [t2]
    s = {}
    while len(l1) != 0:
        t1 = l1.pop()
        t2 = l2.pop()
        if isinstance(t1, Var):
            if t1 == t2:
                continue
            if occurs_check(t1, t2):
                return None
            l1 = [subst(t, {t1.name:t2}) for t in l1]
            l2 = [subst(t, {t1.name:t2}) for t in l2]
            s[t1.name] = t2
        elif isinstance(t2, Var):
            if occurs_check(t2, t1):
                return None
            l1 = [subst(t, {t2.name:t1}) for t in l1]
            l2 = [subst(t, {t2.name:t1}) for t in l2]
            s[t2.name] = t1
        elif isinstance(t1, Fn) and isinstance(t2, Fn):
            if t1.name != t2.name or len(t1.args) != len(t2.args):
                return None
            l1.extend(t1.args)
            l2.extend(t2.args)
        else:
            raise ValueError("Invalid term")
    return s

In [ ]:
# fingerprinting https://link.springer.com/chapter/10.1007/978-3-642-31365-3_37

def symcount(t:Term):
    pass
def varcount():
    pass



In [ ]:
"""
@dataclass
class Literal():
    neg: bool
    term:Term

@dataclass
class Clause():
    literals: list[Literal]
"""
@dataclass
class Clause():
    neg: tuple[Term, ...] # frozenset?
    pos: tuple[Term, ...]
# pos, neg. Would this be better?

#https://github.com/eprover/PyRes/blob/master/resolution.py
def computeFactors(clause):
    res = []
    for poslit in clause.pos:
        for neglit in clause.neg:
            s = mgu(poslit,neglit)
            if s == None:
                continue
            new_clause = Clause(tuple(subst(lit,s) for lit in clause.pos if lit != poslit), tuple(subst(lit,s) for lit in clause.neg if lit != neglit))
            res.append(Clause(new_clause))
    return res

def computeResolvents(clause1: Clause, clause2: Clause):
    res = []
    for lit1 in clause1.pos:
        for lit2 in clause2.neg:
            s = mgu(lit1,lit2)
            if s == None:
                continue
            new_clause = Clause(tuple(subst(lit,s) for lit in clause1.pos if lit != lit1) + tuple(subst(lit,s) for lit in clause2.pos if lit != lit2), tuple(subst(lit,s) for lit in clause1.neg if lit != lit1) + tuple(subst(lit,s) for lit in clause2.neg if lit != lit2))
            res.append(Clause(new_clause))
    return res


def prove(clauses):
    unprocessed = set(clauses)
    processed = set()
    while len(unprocessed) >= 0:
        new = []
        clause = unprocessed.pop()
        new.extend(computeFactors(clause))
        for clause2 in processed:
            new.extend(computeResolvents(clause, clause2))
        processed.add(clause)
        delta = processed.difference(new)
        unprocessed.update(delta)



In [ ]:
# The proof recording system
proof_db = []
def axiom(s):
    proof_db.append((s, "axiom"))
    return len(proof_db)
def formula(s):
    return proof_db[s][0]
def factor(c):
    f = compute_factor(formula(c))
    proof_db.append((f, ("factor",c)))
    return len(proof_db)
def resolve(c1,c2):
    r = compute_resolvent(formula(c1), formula(c2))
    proof_db.append((r, ("resolve",c1,c2)))
    return len(proof_db)







In [ ]:
    for i, lit1 in enumerate(clause.literals):
        for lit2 in clause.literals[i:]:
            if lit1.neg == lit2.neg:
                continue
            s = mgu(lit1,lit2)
            if s == None:
                continue
            new_clause = [Literal(lit.neg, subst(lit.term,s)) for lit in clause.literals if lit != lit1 and lit != lit2]
            res.append(Clause(new_clause))

metitarski. What if I included arb

In [ ]:
@dataclass(frozen=True)
class Fact():
    ctx : tuple[Term, ...]
    fact: Term
class Rule():


def hypo_datalog():
    rules = []
    facts = []
    while True:
        for rule in rules:
            for fact in facts:
                compute_resolvent(rule,fact)

In [ ]:
def prolog():
    rules = []
    

In [ ]:
def datalog():
    rules = []
    facts = []